This is a practice in building various recommedation engine based on collaborative filtering using the movielens movie db

-item-item
-user-user
-user-item
-SVD/matrix transformation


In [2]:
import numpy as np
import pandas as pd
import re
import sklearn as skl
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD

In [3]:

# pass in column names for each CSV and read them using pandas. 
# Column names available in the readme file

#Reading users file:
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.user', sep='|', names=u_cols,
 encoding='latin-1')

#Reading ratings file:
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']
ratings = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.data', sep='\t', names=r_cols,
 encoding='latin-1')

#Reading items file:
i_cols = ['movie_id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\u.item', sep='|', names=i_cols,
 encoding='latin-1')

In [4]:
users.head()

,user_id,age,sex,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


In [5]:
items.head()

,movie_id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [6]:
ratings.head()

,user_id,movie_id,rating,unix_timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [7]:
ratings['rating'].unique()

array([3, 1, 2, 4, 5], dtype=int64)

In [8]:
ratings_base = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv(r'C:\Users\suhaib.qazi\Downloads\ml-100k\ml-100k\ua.test', sep='\t', names=r_cols, encoding='latin-1')

In [9]:
cols = ['movie_id','movie title']
items2 = items[cols]

In [10]:
rating_items = pd.merge(ratings,items2,how='left',on='movie_id')
rating_items = rating_items.drop('unix_timestamp',axis=1)

In [11]:
rating_items

,user_id,movie_id,rating,movie title
0,196,242,3,Kolya (1996)
1,186,302,3,L.A. Confidential (1997)
2,22,377,1,Heavyweights (1994)
3,244,51,2,Legends of the Fall (1994)
4,166,346,1,Jackie Brown (1997)
5,298,474,4,Dr. Strangelove or: How I Learned to Stop Worr...
6,115,265,2,"Hunt for Red October, The (1990)"
7,253,465,5,"Jungle Book, The (1994)"
8,305,451,3,Grease (1978)
9,6,86,3,"Remains of the Day, The (1993)"


In [12]:
review_cnt = rating_items.groupby('movie_id', as_index=False).agg({'rating':'count'})

In [13]:
review_cnt

,movie_id,rating
0,1,452
1,2,131
2,3,90
3,4,209
4,5,86
5,6,26
6,7,392
7,8,219
8,9,299
9,10,89


In [14]:
review_cnt = review_cnt.rename(columns = {'rating':'rating_cnt'})

In [15]:
rating_items2 = pd.merge(rating_items,review_cnt,on='movie_id')

In [16]:
rating_items2['rating_cnt'].quantile(np.arange(0,1,.05))

0.00      1.0
0.05     19.0
0.10     34.0
0.15     46.0
0.20     60.0
0.25     71.0
0.30     85.0
0.35     98.0
0.40    116.0
0.45    128.0
0.50    145.0
0.55    162.0
0.60    174.0
0.65    194.0
0.70    217.0
0.75    239.0
0.80    259.0
0.85    293.0
0.90    331.0
0.95    420.0
Name: rating_cnt, dtype: float64

In [17]:
rating_items3 = rating_items2[rating_items2['rating_cnt'] > 50]

In [18]:
rating_items3

,user_id,movie_id,rating,movie title,rating_cnt
0,196,242,3,Kolya (1996),117
1,63,242,3,Kolya (1996),117
2,226,242,5,Kolya (1996),117
3,154,242,3,Kolya (1996),117
4,306,242,5,Kolya (1996),117
5,296,242,4,Kolya (1996),117
6,34,242,5,Kolya (1996),117
7,271,242,4,Kolya (1996),117
8,201,242,4,Kolya (1996),117
9,209,242,4,Kolya (1996),117


In [19]:
rating_items3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 83215 entries, 0 to 96709
Data columns (total 5 columns):
user_id        83215 non-null int64
movie_id       83215 non-null int64
rating         83215 non-null int64
movie title    83215 non-null object
rating_cnt     83215 non-null int64
dtypes: int64(4), object(1)
memory usage: 3.8+ MB


In [34]:
rating_items_piv = rating_items3.pivot_table(index='movie title',columns='user_id',values='rating', aggfunc='min').fillna(0)

In [36]:
rating_items_piv

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
movie title,,,,,,,,,,,,,,,,,,,,,
101 Dalmatians (1996),2.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,...,2.0,0.0,0.0,2.0,4.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men (1957),5.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,0.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2 Days in the Valley (1996),0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
"20,000 Leagues Under the Sea (1954)",3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2001: A Space Odyssey (1968),4.0,0.0,0.0,0.0,4.0,5.0,5.0,0.0,0.0,5.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
"39 Steps, The (1935)",0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0
Absolute Power (1997),0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0
"Abyss, The (1989)",3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
Ace Ventura: Pet Detective (1994),3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0


In [98]:
rating_items_piv.shape

(592, 943)

In [99]:
X = rating_items_piv.values
X.shape

(592, 943)

In [118]:
SVD = TruncatedSVD(n_components=10, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(592, 10)

In [120]:
import warnings
warnings.filterwarnings("ignore",category =RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(592, 592)

In [121]:

film_cols = rating_items_piv.index
film_list = list(film_cols)
film = 'Star Wars (1977)'
film_idx = rating_items_piv.index.get_loc(film)
print(film_idx)

514


In [122]:
corr_film  = corr[film_idx]

In [123]:
list(film_cols[(corr_film<1.0) & (corr_film>0.9)])

['Independence Day (ID4) (1996)',
 'Raiders of the Lost Ark (1981)',
 'Return of the Jedi (1983)',
 'Star Trek: First Contact (1996)',
 'Toy Story (1995)']